In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
"""Computing the score 
    score(e;q,m) = P (e|q,m) ∝ P (e|m)P (q|e) 
"""
from termcolor import colored
from tabulate import tabulate
import pickle5 as pickle

import time
from entity_linker import EntityLinker
from flair.models import SequenceTagger
from entity_ranking import DictionaryRanking, QueryEntityRanking


In [3]:
#loading dicitonary of commonness,
print('Loading mention2pem dictionary ...')
handle = open('../data/pem/pem.pickle', 'rb')
mention2pem = pickle.load(handle)


print('Loading entity description dictionary ...')
handle_desc = open('../data/pem/entity2description.pickle', 'rb')
entity2description = pickle.load(handle_desc)
print('NUmber of entities: ', len(entity2description))

print('Loading dictionary of term frequency ...')
handle_desc = open('../data/pem/mention_freq.pickle', 'rb')
mention2freq = pickle.load(handle_desc)
print('Number of term in the collection: ', len(mention2freq))

#given by create_term_req
collection_size_terms = len(mention2pem)


Loading mention2pem dictionary ...
Loading entity description dictionary ...
NUmber of entities:  2680002
Loading dictionary of term frequency ...
Number of term in the collection:  3506008


In [4]:
list(mention2pem['reproduction'].keys())

['GO:0000003']

In [5]:
list(mention2pem['reproduction'].values())


[1.0]

In [12]:
entity2description['GO:0005578']


()

In [13]:
entity2description['GO:0031012']


('lying',
 'tissues',
 'support',
 'biochemical',
 'matrisome',
 'structure',
 'external',
 'matrix',
 'extracellular',
 'cells',
 'cues',
 'proteinaceous',
 'structural',
 'biomechanical',
 'provides')

In [4]:
# load the NER tagger
tagger = SequenceTagger.load(
    '../resources/taggers/sota-ner-flair/best-model.pt')


2021-09-08 18:48:29,094 loading file ../resources/taggers/sota-ner-flair/best-model.pt


In [5]:
dictionarysearch_strategy = DictionaryRanking(mention2pem) 
queryranking_strategy = QueryEntityRanking(
    entity2description=entity2description,
    mention_freq=mention2freq,
    mention2pem=mention2pem
    )
e_linker = EntityLinker(
    ranking_strategy=queryranking_strategy,
    ner_model=tagger,
    mention2pem=mention2pem
    )


In [20]:
e_linker.link_entities('a quaternary ammonium salt that is the monoiodide')

Analizing mentions: ['quaternary ammonium salt']


{'quaternary ammonium salt': ('CHEBI:35273', 117.02924314033649)}

In [22]:
e_linker.link_entities('sexual reproduction')

Analizing mentions: ['reproduction']


{'reproduction': ('GO:0000003', 133.7471342435693)}

In [6]:
e_linker.link_entities('Quaternary ammonium salt that is the monoiodide')

quaternary ammonium salt that is the monoiodide
Analizing mentions: ['ammonium salt']


{'ammonium salt': ('CHEBI:47704', 209.34564719620988)}

In [31]:
e_linker.link_entities('Quaternary ammonium salt that is the monoiodide. Sexual reproduction')


Quaternary ammonium salt that is the monoiodide. Sexual reproduction 
Analizing mentions: []


{}

In [32]:
e_linker.link_entities(
    'Quaternary ammonium salt that is the monoiodide that can be found in some species with sexual reproduction reduced')


Quaternary ammonium salt that is the monoiodide that can be found in some species with sexual reproduction reduced
Analizing mentions: []


{}

In [34]:
e_linker.link_entities('A molecular entity having a net electric.')


A molecular entity having a net electric.
Analizing mentions: ['molecular entity']


{'molecular entity': ('CHEBI:23367', 2.667553377723604)}

In [36]:
e_linker.link_entities('theres molecular entity with electric charge')


theres molecular entity with electric charge
Analizing mentions: ['molecular entity']


{'molecular entity': ('CHEBI:23367', 3.245904530608999)}

In [37]:
text = 'The fluoride salt of beryllium (+2 oxidation state). In the solid state it exists as a glass, with four-coordinate Be(2+) tetrahedral centres and two-coordinate fluoride centres. As a gas it adopts a linear triatomic structure and in the liquid state a fluctuating tetrahedral structure. In protein crystallography it is used as a mimic of phosphate.'
e_linker.link_entities(text)

The fluoride salt of beryllium (+2 oxidation state). In the solid state it exists as a glass, with four-coordinate Be(2+) tetrahedral centres and two-coordinate fluoride centres. As a gas it adopts a linear triatomic structure and in the liquid state a fluctuating tetrahedral structure. In protein crystallography it is used as a mimic of phosphate.
Analizing mentions: ['fluoride salt']


{'fluoride salt': ('CHEBI:24060', 2.216991408264466)}

In [38]:
e_linker.link_entities('The fluoride salt of beryllium (+2 oxidation state)')


The fluoride salt of beryllium (+2 oxidation state)
Analizing mentions: ['fluoride salt']


{'fluoride salt': ('CHEBI:24060', 24.157725392041723)}

In [40]:
e_linker.link_entities(
    'In the solid state it exists as a glass, with four-coordinate Be(2+) tetrahedral centres and two-coordinate fluoride centres.')


In the solid state it exists as a glass, with four-coordinate Be(2+) tetrahedral centres and two-coordinate fluoride centres.
Analizing mentions: []


{}

In [41]:
e_linker.link_entities(
    ' As a gas it adopts a linear triatomic structure and in the liquid state a fluctuating tetrahedral structure.')


 As a gas it adopts a linear triatomic structure and in the liquid state a fluctuating tetrahedral structure.
Analizing mentions: []


{}

In [43]:
e_linker.link_entities('In protein crystallography it is used as a mimic of phosphate.')

In protein crystallography it is used as a mimic of phosphate.
Analizing mentions: []


{}